Now we arrived at the best part of this course which is pretrain model and transfer learning so we;re going to learn the code syntax and the theory behinde this fact in order to be ready work with small dataset and avoid overfitting.


The Step we follow to use pre train model is like below:



1.   Download the weight of pretrain model
2.   Choose the pretrain model for load the weight on it
3.   Freeze the first Layer
4.   Give the output of last layer
5.   create our DNN model and attach to the end of the model
6.   Now we preprocess our input and then pass through the model to learn



In [1]:
# Download The weight of pretrain model

!!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

['--2021-06-05 04:51:58--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.253.63.128, 142.250.31.128, ...',
 'Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.',
 'HTTP request sent, awaiting response... 200 OK',
 'Length: 87910968 (84M) [application/x-hdf]',
 'Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’',
 '',
 '',
 '          /tmp/ince   0%[                    ]       0  --.-KB/s               ',
 '         /tmp/incep  21%[===>                ]  17.66M  88.3MB/s               ',
 '        /tmp/incept  63%[===========>        ]  53.06M   133MB/s               ',
 '/tmp/inception_v3_w 100%[===================>]  83.84M   169MB/s    in 0.5s    ',
 '',
 '2021-06-05 04:51:58 (169 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/879109

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [3]:
local_weights_files = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Create Our Inception Moodel
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights=None
)

In [4]:
# Load weights
pre_trained_model.load_weights(local_weights_files)

In [5]:
# Now We need to freeze all models

for layer in pre_trained_model.layers:
    layer.trainable=False

In [6]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [7]:
last_layer = pre_trained_model.get_layer('mixed7')
print('OutPut Shape : {}'.format(last_layer.output_shape))

OutPut Shape : (None, 7, 7, 768)


In [8]:
last_output = last_layer.output

Now We Need to Define our DNN Model to add to end of the pre-train model.

In [9]:
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop

In [10]:
x = layers.Flatten()(last_output)
x = layers.Dense(units=1024, activation='relu')(x)
x = layers.Dropout(rate=0.2)(x)
x = layers.Dense(units=1, activation='sigmoid')(x)

In [11]:
model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Now Preprocess the photo and start to fit on our data

In [12]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

--2021-06-05 04:52:07--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.128, 172.253.122.128, 172.253.63.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  38.0MB/s    in 1.7s    

2021-06-05 04:52:09 (38.0 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [13]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 10,
            validation_steps = 50,
            verbose = 2)

Epoch 1/10
100/100 - 50s - loss: 0.3745 - accuracy: 0.8505 - val_loss: 0.1395 - val_accuracy: 0.9430
Epoch 2/10
100/100 - 17s - loss: 0.2044 - accuracy: 0.9155 - val_loss: 0.1680 - val_accuracy: 0.9360
Epoch 3/10
100/100 - 17s - loss: 0.1993 - accuracy: 0.9270 - val_loss: 0.1007 - val_accuracy: 0.9640
Epoch 4/10
100/100 - 17s - loss: 0.2272 - accuracy: 0.9280 - val_loss: 0.1594 - val_accuracy: 0.9470
Epoch 5/10
100/100 - 17s - loss: 0.1735 - accuracy: 0.9340 - val_loss: 0.1088 - val_accuracy: 0.9630
Epoch 6/10
100/100 - 17s - loss: 0.1881 - accuracy: 0.9305 - val_loss: 0.1496 - val_accuracy: 0.9560
Epoch 7/10
100/100 - 17s - loss: 0.1863 - accuracy: 0.9360 - val_loss: 0.1085 - val_accuracy: 0.9660
Epoch 8/10
100/100 - 17s - loss: 0.1791 - accuracy: 0.9410 - val_loss: 0.1059 - val_accuracy: 0.9640
Epoch 9/10
100/100 - 17s - loss: 0.1615 - accuracy: 0.9445 - val_loss: 0.1050 - val_accuracy: 0.9690
Epoch 10/10
100/100 - 16s - loss: 0.1603 - accuracy: 0.9520 - val_loss: 0.1537 - val_accura